In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
def resumen(datos):
    return print('Forma:\n', datos.shape,'\n\n',
                 'Columnas:\n', datos.columns,'\n\n',
                 'Tipos:\n', datos.dtypes,'\n\n',
                 'Datos nulos:\n', datos.isnull().sum(),'\n\n',
                 'Cabecera:\n', datos.head(10),'\n\n',
                 'Últimos datos:\n', datos.tail(10),'\n\n',
                 'Estadísticos:\n', datos.describe([0.05,0.25,0.5,0.75,0.95])
                )
def truncar(numero, decimales = 0):
    return np.trunc(numero*10**decimales)/(10**decimales)

In [3]:
def metadata(ejex, ejey):
    # plt.title(titulo, fontsize = 60, fontweight='bold')
    plt.xlabel(ejex, fontsize = 35)
    plt.ylabel(ejey, fontsize = 35)
    plt.xticks(fontsize = 25)
    plt.yticks(fontsize = 25)
    return
def cifras_signif(i, cifras = 3):
    texto = str('{:g}'.format(float('{:.{p}g}'.format(i, p = cifras))))
    return texto

In [4]:
def gpp(par, alfa, optimo):
    gpp = -(alfa*par*optimo) / (alfa*par+optimo)
    return gpp
def lloyd_taylor(t, ref, e0):
    # Poner t en centigrados
    t0 = 269
    resp = ref * np.exp(-e0/(t+273.16-t0))
    return resp
def q10(t, ref, q10):
    # Poner t en centigrados
    tref = 283.16
    resp = ref * q10**((t+273.16-tref)/10)
    return resp

In [5]:
def err_gpp_par(par, alfa, optimo, error_porc):
    error = (gpp(par, alfa, optimo)**2
             /(alfa*par)
             *error_porc
            )
    return error
def err_gpp_alfa(par, alfa, optimo, error_alfa):
    captura = gpp(par, alfa, optimo)
    error = ((1
              - captura/optimo
             )
             * captura
             / alfa
             * error_alfa
            )
    return error
def err_gpp_opt(par, alfa, optimo, error_opt):
    captura = gpp(par, alfa, optimo)
    error = ((1
              - captura/(alfa*par)
             )
             * captura
             / optimo
             * error_opt
            )
    return error
def err_reco_temp(t, ref, e0, err_t):
    # Poner t en centigrados
    t0 = 267
    error = (lloyd_taylor(t, ref, e0)
             *e0
             *err_t
             /(t+273-t0)**2
            )
    return error
def err_reco_ref(t, ref, e0, err_ref):
    # Poner t en centigrados
    t0 = 267
    error = lloyd_taylor(t, ref, e0)*err_ref/ref
    return error
def err_reco_e0(t, ref, e0, err_e0):
    # Poner t en centigrados
    t0 = 267
    error = -lloyd_taylor(t, ref, e0)*err_e0/(t+273-t0)
    return error
def err_q10_temp(t, ref, Q10, err_t):
    # Poner t en centigrados
    error = (q10(t, ref, Q10)
             *np.log(Q10)
             *err_t
             /10
            )
    return error
def err_q10_ref(t, ref, Q10, err_ref):
    # Poner t en centigrados
    error = q10(t, ref, Q10)/ref*err_ref
    return error
def err_q10_q(t, ref, Q10, err_q):
    # Poner t en centigrados
    error = q10(t, ref, Q10)*err_q*(t-10)/(10*Q10)
    return error

In [6]:
anu = pd.read_csv('C:\\Users\\nahue\\Desktop\\Tesis_2\\Datos\\Relleno_anual.txt',
                        #'C:\\Users\\BIOMET\\Desktop\\Tesis_2\\Datos\\Relleno_anual.txt',
                        delimiter = '\t',
                        decimal = '.',
                        na_values = -9999,
                        encoding = 'ascii'
                        )
lut = pd.read_csv('C:\\Users\\nahue\\Desktop\\Tesis_2\\Datos\\Relleno_LUT.txt',
                        #'C:\\Users\\BIOMET\\Desktop\\Tesis_2\\Datos\\Relleno_LUT.txt',
                        delimiter = '\t',
                        decimal = '.',
                        na_values = -9999,
                        encoding = 'ascii'
                        )
reddy = pd.read_csv('C:\\Users\\nahue\\Desktop\\Tesis_2\\Datos\\output.txt',
                        #'C:\\Users\\BIOMET\\Desktop\\Tesis_2\\Datos\\output.txt',
                        delimiter = '\t',
                        decimal = '.',
                        na_values = -9999,
                        encoding = 'ascii',
                        skiprows = [1]                        
                        )
datos = pd.read_csv('C:\\Users\\nahue\\Desktop\\Tesis_2\\Datos\\Completos_buenos.txt',
                    #'C:\\Users\\BIOMET\\Desktop\\Tesis_2\\Datos\\Completos_buenos.txt',
                    delimiter = '\t',
                    decimal = '.',
                    na_values = -9999,
                    skiprows = [1],
                    encoding = 'ascii'
                    )

In [7]:
resumen(datos)

Forma:
 (14956, 38) 

 Columnas:
 Index(['Year', 'DoY', 'Hour', 'NEE', 'LE', 'H', 'Rg', 'Tair', 'Tsoil', 'rH',
       'VPD', 'Ustar', 'TS1', 'TS2', 'TS3', 'TS4', 'U', 'DIR', 'PP', 'QG1',
       'QG2', 'WET', 'PAR', 'TSH1', 'HS1', 'GH1', 'TSH2', 'HS2', 'GH2', 'TSH3',
       'HS3', 'GH3', 'TSH4', 'HS4', 'GH4', 'PAR.1', '   r_err_co2[%]',
       '   noise_co2[%]'],
      dtype='object') 

 Tipos:
 Year               float64
DoY                float64
Hour               float64
NEE                float64
LE                 float64
H                  float64
Rg                 float64
Tair               float64
Tsoil              float64
rH                 float64
VPD                float64
Ustar              float64
TS1                float64
TS2                float64
TS3                float64
TS4                float64
U                  float64
DIR                float64
PP                 float64
QG1                float64
QG2                float64
WET                float64
PAR     

In [8]:
datos['Error porcent'] = datos['   r_err_co2[%]'] + datos['   noise_co2[%]']
datos['Error'] = abs(datos['Error porcent']/100 * reddy['NEE'])
datos_validos = datos[((datos['Rg'] >= 5)
                      & (datos['Ustar'] > 0.14167)
                      & (datos['PAR.1'] > 1)
                      & pd.notna(datos['NEE'])
                      & (datos['Error porcent'] < 30)
                     )
                    | ((datos['Rg'] < 5)
                      & (datos['Ustar'] > 0.14167)
                      & pd.notna(datos['Tair'])
                      & (datos['NEE'] >= 0)
                      & (datos['Error porcent'] < 40)
                      & (datos['Error'] < 0.5)
                     )
                     ]

In [9]:
comparacion = reddy[['DoY', 'Hour', 'Tair', 'Tsoil', 'NEE_f', 'NEE_fsd']][:]
comparacion.head()

,DoY,Hour,Tair,Tsoil,NEE_f,NEE_fsd
0,54,9.0,19.180,18.140,-5.630867,0.955625
1,54,9.5,19.290,18.180,-4.823325,0.132194
2,54,10.0,19.515,18.370,-6.212467,1.220959
3,54,10.5,19.640,19.010,-3.625855,2.624397
4,54,11.0,19.685,19.305,-7.545225,2.351943


In [10]:
comparacion['Std LUT'] = lut['Std LUT'][:]
comparacion['Std LUT MDV'] = lut['Std MDV'][:]
comparacion['Std anu MDV'] = anu['Std MDV'][:]
comparacion['Error turb'] = datos_validos['Error'][:]
comparacion['PAR'] = datos['PAR.1'][:]
comparacion['Dias'] = comparacion.index/48 + 54
comparacion.head()

,DoY,Hour,Tair,Tsoil,NEE_f,NEE_fsd,Std LUT,Std LUT MDV,Std anu MDV,Error turb,PAR,Dias
0,54,9.0,19.180,18.140,-5.630867,0.955625,2.853516,NaN,NaN,NaN,892.5,54.000000
1,54,9.5,19.290,18.180,-4.823325,0.132194,2.224728,NaN,NaN,NaN,797.6,54.020833
2,54,10.0,19.515,18.370,-6.212467,1.220959,4.520234,NaN,NaN,NaN,1218.0,54.041667
3,54,10.5,19.640,19.010,-3.625855,2.624397,4.180161,NaN,NaN,NaN,1070.0,54.062500
4,54,11.0,19.685,19.305,-7.545225,2.351943,3.738405,NaN,NaN,NaN,1463.0,54.083333


In [11]:
comparacion.tail()

,DoY,Hour,Tair,Tsoil,NEE_f,NEE_fsd,Std LUT,Std LUT MDV,Std anu MDV,Error turb,PAR,Dias
14951,365,20.5,21.075,19.8525,3.862050,4.042071,3.026112,NaN,NaN,NaN,0.210,365.479167
14952,365,21.0,20.785,19.7000,3.876199,4.073495,3.026112,NaN,NaN,NaN,11.950,365.500000
14953,365,21.5,19.410,19.5750,3.890915,4.105639,3.026112,NaN,NaN,NaN,12.255,365.520833
14954,365,22.0,18.955,19.4600,3.911156,4.137216,3.026112,NaN,NaN,NaN,21.390,365.541667
14955,365,22.5,18.610,19.3625,3.925391,4.171247,3.026112,NaN,NaN,NaN,5.980,365.562500


In [12]:
alfa_anu = [0.00897, 0.000322]
optimo_anu = [20.4, 1.36]
ref_anu_soil = [19.4, 6.67]
e0_anu_soil = [46, 12.9]
t0_anu_soil = [267, 2.58]
ref_anu_air = [13.7, 3.49]
e0_anu_air = [45.4, 11.2]
to_anu_air = [264, 2.64]
valor_q10 = [1.15, 0.113]
ref_q10 = [3.32, 0.0221]
nans = np.empty(len(comparacion['Dias']))
nans[:] = np.nan
comparacion['Error nee bim'] = nans[:]
ref_bim = {'Valores': [9.59, 9.59, 7.77, 13.29, 7.67, 16.55, 16.55],
           'Std': [5.3, 5.3, 2.88, 4.51, 1.73, 3.8, 3.8]
          }
e0_bim = {'Valores': [23.78, 23.78, 31.08, 40.49, 29.63, 42.84, 42.84],
         'Std': [12.53, 12.53, 5.73, 5.11, 4.1, 5.08, 5.08]
         }
alfa_bim = {'Valores': [0.139, 0.139, 0.027, 0.033, 0.016, 0.024, 0.024],
           'Std': [0.042, 0.042, 0.004, 0.005, 0.001, 0.005, 0.005]
           }
optimo_bim = {'Valores': [9.69, 9.69, 5.66, 3.32, 7.12, 7.31, 7.31],
              'Std': [0.688, 0.688, 0.284, 0.110, 0.265, 0.277, 0.277]
             }
comparacion['Err q10 temp'] = err_q10_temp(comparacion['Tsoil'],
                                 ref_q10[0],
                                 valor_q10[0],
                                 0.06
                                )
comparacion['Err q10 ref'] = err_q10_ref(comparacion['Tsoil'],
                                 ref_q10[0],
                                 valor_q10[0],
                                 ref_q10[1]
                                )
comparacion['Err q10 q'] = err_q10_q(comparacion['Tsoil'],
                                 ref_q10[0],
                                 valor_q10[0],
                                 valor_q10[1]
                                )
comparacion['Err q10 final'] = np.sqrt(comparacion['Err q10 temp']**2
                                       + comparacion['Err q10 ref']**2
                                       + comparacion['Err q10 q']**2
                                       )
comparacion['Err gpp anu par'] = err_gpp_par(comparacion['PAR'],
                                     alfa_anu[0],
                                     optimo_anu[0],
                                     0.027
                                    )
comparacion['Err gpp anu alfa'] = err_gpp_alfa(comparacion['PAR'],
                                     alfa_anu[0],
                                     optimo_anu[0],
                                     alfa_anu[1]
                                    )
comparacion['Err gpp anu opt'] = err_gpp_opt(comparacion['PAR'],
                                     alfa_anu[0],
                                     optimo_anu[0],
                                     optimo_anu[1]
                                    )
comparacion['Err gpp final'] = np.sqrt(comparacion['Err gpp anu par']**2
                                       + comparacion['Err gpp anu alfa']**2
                                       + comparacion['Err gpp anu opt']**2
                                       )
comparacion['Err reco anu temp'] = err_reco_temp(comparacion['Tsoil'],
                                       ref_anu_soil[0],
                                       e0_anu_soil[0],
                                       0.06
                                       )
comparacion['Err reco anu ref'] = err_reco_ref(comparacion['Tsoil'],
                                       ref_anu_soil[0],
                                       e0_anu_soil[0],
                                       ref_anu_soil[1]
                                       )
comparacion['Err reco anu e0'] = err_reco_e0(comparacion['Tsoil'],
                                       ref_anu_soil[0],
                                       e0_anu_soil[0],
                                       e0_anu_soil[1]
                                       )
comparacion['Err reco soil final'] = np.sqrt(comparacion['Err reco anu temp']**2
                                       + comparacion['Err reco anu ref']**2
                                       + comparacion['Err reco anu e0']**2
                                       )
comparacion['Err reco air temp'] = err_reco_temp(comparacion['Tair'],
                                       ref_anu_air[0],
                                       e0_anu_air[0],
                                       0.2
                                       )
comparacion['Err reco air ref'] = err_reco_ref(comparacion['Tair'],
                                       ref_anu_air[0],
                                       e0_anu_air[0],
                                       ref_anu_air[1]
                                       )
comparacion['Err reco air e0'] = err_reco_e0(comparacion['Tair'],
                                       ref_anu_air[0],
                                       e0_anu_air[0],
                                       e0_anu_air[1]
                                       )
comparacion['Err reco air final'] = np.sqrt(comparacion['Err reco air temp']**2
                                       + comparacion['Err reco air ref']**2
                                       + comparacion['Err reco air e0']**2
                                       )
comparacion['Err nee air'] = (comparacion['Err gpp final']
                              + comparacion['Err reco air final']
                             )
comparacion['Err nee soil'] = (comparacion['Err gpp final']
                               + comparacion['Err reco soil final']
                              )
dias = np.arange(0, 480, 60)
for i in range(1, len(dias)):
    desde = dias[i-1]
    hasta = dias[i]
    datos_bimes = comparacion[(comparacion['Dias'] >= desde)
                              & (comparacion['Dias'] < hasta)
                              ][:]
    datos_bimes['Err reco temp'] = err_reco_temp(datos_bimes['Tsoil'],
                                       ref_bim['Valores'][i-1],
                                       e0_bim['Valores'][i-1],
                                       0.06
                                       )
    datos_bimes['Err reco ref'] = err_reco_ref(datos_bimes['Tsoil'],
                                       ref_bim['Valores'][i-1],
                                       e0_bim['Valores'][i-1],
                                       ref_bim['Std'][i-1]
                                       )
    datos_bimes['Err reco e0'] = err_reco_e0(datos_bimes['Tsoil'],
                                       ref_bim['Valores'][i-1],
                                       e0_bim['Valores'][i-1],
                                       e0_bim['Std'][i-1]
                                       )
    datos_bimes['Err Gpp par'] = err_gpp_par(datos_bimes['PAR'],
                                     alfa_bim['Valores'][i-1],
                                     optimo_bim['Valores'][i-1],
                                     0.027
                                    )
    datos_bimes['Err Gpp alfa'] = err_gpp_alfa(datos_bimes['PAR'],
                                     alfa_bim['Valores'][i-1],
                                     optimo_bim['Valores'][i-1],
                                     alfa_bim['Std'][i-1]
                                    )
    datos_bimes['Err Gpp optimo'] = err_gpp_opt(datos_bimes['PAR'],
                                     alfa_bim['Valores'][i-1],
                                     optimo_bim['Valores'][i-1],
                                     optimo_bim['Std'][i-1]
                                    )
    datos_bimes['Err reco'] = np.sqrt(datos_bimes['Err reco temp']**2
                                       + datos_bimes['Err reco ref']**2
                                       + datos_bimes['Err reco e0']**2
                                       )
    datos_bimes['Err Gpp'] = np.sqrt(datos_bimes['Err Gpp par']**2
                                       + datos_bimes['Err Gpp alfa']**2
                                       + datos_bimes['Err Gpp optimo']**2
                                       )
    datos_bimes['Err nee'] = (datos_bimes['Err Gpp']
                              + datos_bimes['Err reco']
                              )
    comparacion['Error nee bim'].loc[
        datos_bimes.index] = datos_bimes['Err nee'][:]

C:\Users\nahue\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
dias = [0, 120, 180, 240, 300, 366]
error_anu = list(range(1, len(dias)))
error_bim = list(range(1, len(dias)))
error_tair = list(range(1, len(dias)))
error_lut = list(range(1, len(dias)))
error_reddy = list(range(1, len(dias)))
for j in ['Err nee soil', 'Err nee air', 'Error nee bim', 'Std LUT', 'NEE_fsd']:
    for i in list(range(1, len(dias))):
        desde = dias[i-1]
        hasta = dias[i]
        datos_bimes = comparacion[(comparacion['Dias'] >= desde)
                                  & (comparacion['Dias'] < hasta)
                                  ][:]
        if j == 'Std LUT':
            datos_bimes['Error turb'] = datos_bimes['Error turb'].fillna(
                                                    datos_bimes[j]
                                                    ).fillna(
                                            datos_bimes['Std LUT MDV']
                                            )
            datos_bimes['Error turb'] = datos_bimes['Error turb']*30*60*44/1000000
            error_lut[i-1] = np.sqrt((datos_bimes['Error turb']**2).sum())
        elif j == 'NEE_fsd':
            datos_bimes['NEE_fsd'] = datos_bimes['NEE_fsd']*30*60*44/1000000
            error_reddy[i-1] = np.sqrt((datos_bimes['NEE_fsd']**2).sum())
        else:
            datos_bimes['Error turb'] = datos_bimes['Error turb'].fillna(
                                                    datos_bimes[j]
                                                    ).fillna(
                                            datos_bimes['Std anu MDV']
                                            )
            datos_bimes['Error turb'] = datos_bimes['Error turb']*30*60*44/1000000
            if j == 'Err nee soil':
                error_anu[i-1] = np.sqrt((datos_bimes['Error turb']**2).sum())
            if j == 'Err nee air':
                error_tair[i-1] = np.sqrt((datos_bimes['Error turb']**2).sum())
            if j == 'Error nee bim':
                error_bim[i-1] = np.sqrt((datos_bimes['Error turb']**2).sum())

In [14]:
total_anu = np.linalg.norm(error_anu)*1000/100000
total_bim = np.linalg.norm(error_bim)*1000/100000
total_air = np.linalg.norm(error_tair)*1000/100000
total_lut = np.linalg.norm(error_lut)*1000/100000
total_reddy = np.linalg.norm(error_reddy)*1000/100000

In [15]:
error_reddy

[8.487579118218171,
 5.2888274638944415,
 4.16089977135084,
 5.69227141842297,
 11.728524139301877]

In [17]:
error_lut

[11.180641068331909,
 5.493259796269121,
 3.410583067550295,
 3.902624295987952,
 8.286708614975092]

In [18]:
error_bim # Gramos de carbono por metro cuadrado

[19.751887804197835,
 3.119444645001502,
 1.9616492519680955,
 1.6756706061312634,
 5.3427923711696375]

In [19]:
error_anu # Gramos de carbono por metro cuadrado

[7.393650063442492,
 3.6931379519182195,
 2.0782944532495335,
 2.686007694103542,
 5.756764961789027]

In [16]:
total_reddy

0.1694945469698853

In [20]:
total_lut

0.1583397394256967

In [21]:
total_anu # Toneladas por hectarea 

0.10629187042526576

In [22]:
total_bim # Toneladas por hectarea 

0.2085831673007857

In [54]:
comparacion['Err nee soil'].mean() # Micromol por metro cuadrado y segundo

1.2479689407961074

In [55]:
comparacion['Err gpp final'].mean()

0.24222964367134914

In [56]:
comparacion['Err reco soil final'].mean()

1.017484829592306

In [57]:
comparacion['Err reco anu temp'].describe()

count    13973.000000
mean         0.009292
std          0.002582
min          0.000424
25%          0.007494
50%          0.010343
75%          0.011449
max          0.011806
Name: Err reco anu temp, dtype: float64

In [58]:
comparacion['Err reco anu ref'].describe()

count    13973.000000
mean         0.498461
std          0.306968
min          0.003275
25%          0.218314
50%          0.482897
75%          0.734463
max          1.518807
Name: Err reco anu ref, dtype: float64

In [24]:
comparacion['Err reco anu e0'].describe()

count    13973.000000
mean        -0.883109
std          0.394739
min         -1.730651
25%         -1.214510
50%         -0.935869
75%         -0.535636
max         -0.015599
Name: Err reco anu e0, dtype: float64

In [32]:
comparacion['Err q10 ref'].describe()

count    13973.000000
mean         0.023144
std          0.001471
min          0.019728
25%          0.021883
50%          0.023163
75%          0.024266
max          0.027999
Name: Err q10 ref, dtype: float64

In [17]:
comparacion['NEE_fsd'].describe()

count    14956.000000
mean         1.457879
std          0.967956
min          0.002334
25%          0.816393
50%          1.232008
75%          1.864009
max          9.520859
Name: NEE_fsd, dtype: float64